In [1]:
import os
import sys
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from triplet_image_loader import TripletImageLoader
from tripletnet import CS_Tripletnet
from visdom import Visdom
import numpy as np
import Resnet_18
from csn import ConditionalSimNet

In [34]:
checkpoint = torch.load(r'C:\Users\User\VSCode\Pytorch_Lrarning\conditional-similarity-networks\runs\test\checkpoint.pth.tar')


In [39]:
checkpoint.keys()

dict_keys(['epoch', 'state_dict', 'best_prec1', 'acc', 'loss'])

In [29]:
a.keys()

odict_keys(['embeddingnet.embeddingnet.conv1.weight', 'embeddingnet.embeddingnet.bn1.weight', 'embeddingnet.embeddingnet.bn1.bias', 'embeddingnet.embeddingnet.bn1.running_mean', 'embeddingnet.embeddingnet.bn1.running_var', 'embeddingnet.embeddingnet.bn1.num_batches_tracked', 'embeddingnet.embeddingnet.layer1.0.conv1.weight', 'embeddingnet.embeddingnet.layer1.0.bn1.weight', 'embeddingnet.embeddingnet.layer1.0.bn1.bias', 'embeddingnet.embeddingnet.layer1.0.bn1.running_mean', 'embeddingnet.embeddingnet.layer1.0.bn1.running_var', 'embeddingnet.embeddingnet.layer1.0.bn1.num_batches_tracked', 'embeddingnet.embeddingnet.layer1.0.conv2.weight', 'embeddingnet.embeddingnet.layer1.0.bn2.weight', 'embeddingnet.embeddingnet.layer1.0.bn2.bias', 'embeddingnet.embeddingnet.layer1.0.bn2.running_mean', 'embeddingnet.embeddingnet.layer1.0.bn2.running_var', 'embeddingnet.embeddingnet.layer1.0.bn2.num_batches_tracked', 'embeddingnet.embeddingnet.layer1.1.conv1.weight', 'embeddingnet.embeddingnet.layer1.1.b

In [16]:
import scipy.stats as stats

 
# Creating data groups
data_group1 = np.array([1.92, 12.12, 13.16, 8.57, 2.56, 13.79])
data_group2 = np.array([0.00, 3.70, 0.00, 4.17, 0.00, 0.00])
 
# Print the variance of both data groups
print(stats.ttest_rel(data_group1, data_group2))



Ttest_relResult(statistic=3.4448283904160903, pvalue=0.018339599077846956)


Fashion Data Triplet

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os # accessing directory structure
import random

In [2]:
DATASET_PATH = r"C:\Users\User\VSCode\Pytorch_Lrarning\conditional-similarity-networks\data_new\FashionProduct\myntradataset"
print(os.listdir(DATASET_PATH))



['images', 'styles.csv']


In [3]:
df = pd.read_csv(DATASET_PATH + "\styles.csv", error_bad_lines=False)
df['image'] = df.apply(lambda row: str(row['id']) + ".jpg", axis=1)

C:\Users\User\AppData\Local\Temp\ipykernel_23664\1253033299.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(DATASET_PATH + "\styles.csv", error_bad_lines=False)
b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields,

In [4]:
df.columns

Index(['id', 'gender', 'masterCategory', 'subCategory', 'articleType',
       'baseColour', 'season', 'year', 'usage', 'productDisplayName', 'image'],
      dtype='object')

In [38]:
df.masterCategory.value_counts()

Apparel           21397
Accessories       11274
Footwear           9219
Personal Care      2403
Free Items          105
Sporting Goods       25
Home                  1
Name: masterCategory, dtype: int64

In [5]:
apparel = df[df.masterCategory == 'Apparel']
apparel

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011.0,Casual,Inkfruit Mens Chain Reaction T-shirt,1855.jpg
...,...,...,...,...,...,...,...,...,...,...,...
44414,30614,Men,Apparel,Topwear,Tshirts,Black,Summer,2012.0,Sports,Nike Men Striped Black Jersey,30614.jpg
44415,13496,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Chimp Men Teja Main Hoon Blue Tshirts,13496.jpg
44417,12544,Women,Apparel,Topwear,Tshirts,Peach,Fall,2011.0,Casual,Tantra Women Printed Peach T-shirt,12544.jpg
44418,42234,Women,Apparel,Topwear,Tops,Blue,Summer,2012.0,Casual,Sepia Women Blue Printed Top,42234.jpg


In [142]:
apparel.image

0        15970.jpg
1        39386.jpg
3        21379.jpg
4        53759.jpg
5         1855.jpg
           ...    
44414    30614.jpg
44415    13496.jpg
44417    12544.jpg
44418    42234.jpg
44421    18842.jpg
Name: image, Length: 21291, dtype: object

In [122]:
apparel.columns

Index(['id', 'gender', 'masterCategory', 'subCategory', 'articleType',
       'baseColour', 'season', 'year', 'usage', 'productDisplayName', 'image'],
      dtype='object')

In [6]:
apparel = apparel[apparel.subCategory != 'Apparel Set']

In [91]:
apparel.gender.value_counts()

Men       11352
Women      8531
Boys        750
Girls       564
Unisex       94
Name: gender, dtype: int64

In [48]:
## Random Triplet Sampling

# Category
CategoryTriplet = []

for c in apparel.subCategory.value_counts().index:
    category = list(apparel[apparel.subCategory == c].id.values)

    for i in range(int(len(category)/2)):

        temp = random.sample(category, 2) 
        category.remove(temp[0])
        category.remove(temp[1])
        temp.insert(1, int(apparel[apparel.subCategory != c].sample(n=1).id.values))  

        CategoryTriplet.append(temp)
        



In [56]:
def Triplet(condtiion):

    Triplet = []

    for c in apparel[condtiion].value_counts().index:
        category = list(apparel[apparel[condtiion] == c].id.values)

        for i in range(int(len(category)/2)):

            temp = random.sample(category, 2) 
            category.remove(temp[0])
            category.remove(temp[1])
            temp.insert(1, int(apparel[apparel[condtiion] != c].sample(n=1).id.values))  

            Triplet.append(temp)
    
    return pd.DataFrame(Triplet)

In [143]:
def Triplet_replacement(condtiion, sample_num):

    Triplet = []

    for c in apparel[condtiion].value_counts().index:
        category = list(apparel[apparel[condtiion] == c].id.values)

        for i in range(sample_num // len(apparel[condtiion].value_counts().index)):

            temp = random.sample(category, 1) 
            temp.insert(1, int(apparel[apparel[condtiion] != c].sample(n=1).id.values))
            temp.insert(2, random.choice(category))  

            Triplet.append(temp)
    
    return pd.DataFrame(Triplet)

In [99]:
Condition = ['gender', 'subCategory', 'articleType', 'baseColour', 'season', 'usage']
Train = ['gender_train', 'category_train', 'articleType_train', 'baseColour_train', 'season_train', 'usage_train']


season_train = Triplet_replacement('season', 20000)
season_val = Triplet_replacement('season', 20000)
season_test = Triplet_replacement('season', 40000)

In [103]:
np.savetxt(r'C:\Users\User\VSCode\Pytorch_Lrarning\conditional-similarity-networks\data_new\tripletlists\season_train.txt', season_train.values, fmt='%d')
np.savetxt(r'C:\Users\User\VSCode\Pytorch_Lrarning\conditional-similarity-networks\data_new\tripletlists\season_val.txt', season_val.values, fmt='%d')
np.savetxt(r'C:\Users\User\VSCode\Pytorch_Lrarning\conditional-similarity-networks\data_new\tripletlists\season_test.txt', season_test.values, fmt='%d')


In [144]:
gender_train = Triplet_replacement('gender', 20000)
gender_val = Triplet_replacement('gender', 20000)
gender_test = Triplet_replacement('gender', 40000)

In [90]:
print(apparel[apparel.id == 19630].season)
print(apparel[apparel.id == 6038].season)
apparel[apparel.id == 52486].season

8311    Summer
Name: season, dtype: object
17067    Summer
Name: season, dtype: object


195    Winter
Name: season, dtype: object

In [100]:
season_train.values

array([[50681,  9711, 56287],
       [27139, 14622, 57022],
       [ 5113, 20357, 38973],
       ...,
       [23627, 13106,  2126],
       [26515, 10486,  2147],
       [28549, 22205, 39810]], dtype=int64)

'c:\\Users\\User\\VSCode\\Pytorch_Lrarning\\conditional-similarity-networks_new'